# Lab 04-01 : Multivariate Linear Regression

In [54]:
import torch
import torch.nn as nn             # 학습 모델 저장 모듈
import torch.nn.functional as F   # 손실함수 저장 모듈
import torch.optim as optim       # 최적화 함수 모듈

In [55]:
# random seed 고정
torch.manual_seed(1)

## 1) 행렬 연산 고려 X

In [4]:
# 데이터 정의, feature가 3개
x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [5]:
# 모델 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정 -> 경사하강법 이용
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} w1: {:.3f} w2: {:.3f} w3: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, w1.item(), w3.item(), w3.item(), b.item(), cost.item()
        ))

Epoch    0/1000 w1: 0.294 w2: 0.297 w3: 0.297 b: 0.003 Cost: 29661.800781
Epoch  100/1000 w1: 0.674 w2: 0.676 w3: 0.676 b: 0.008 Cost: 1.563628
Epoch  200/1000 w1: 0.679 w2: 0.677 w3: 0.677 b: 0.008 Cost: 1.497595
Epoch  300/1000 w1: 0.684 w2: 0.677 w3: 0.677 b: 0.008 Cost: 1.435044
Epoch  400/1000 w1: 0.689 w2: 0.678 w3: 0.678 b: 0.008 Cost: 1.375726
Epoch  500/1000 w1: 0.694 w2: 0.678 w3: 0.678 b: 0.009 Cost: 1.319507
Epoch  600/1000 w1: 0.699 w2: 0.679 w3: 0.679 b: 0.009 Cost: 1.266222
Epoch  700/1000 w1: 0.704 w2: 0.679 w3: 0.679 b: 0.009 Cost: 1.215703
Epoch  800/1000 w1: 0.709 w2: 0.679 w3: 0.679 b: 0.009 Cost: 1.167810
Epoch  900/1000 w1: 0.713 w2: 0.680 w3: 0.680 b: 0.009 Cost: 1.122429
Epoch 1000/1000 w1: 0.718 w2: 0.680 w3: 0.680 b: 0.009 Cost: 1.079390


## 2) 행렬 연산 고려 O

- 독립 변수의 수를 후에 추가적으로 늘리거나 줄이더라도 가설 선언 코드를 수정할 필요 없음

In [6]:
# x_train = (5 x 3) 행렬
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

print(x_train.shape)
print(y_train.shape)

torch.Size([5, 3])
torch.Size([5, 1])


In [7]:
# 모델 초기화
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):

    # H(x) 계산 -> matmul() 이용
    hypothesis = x_train.matmul(W) + b # or .mm or @

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost: 9298.520508
Epoch    2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost: 2915.712402
Epoch    3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost: 915.040527
Epoch    4/20 hypothesis: tensor([137.7967, 165.6247, 163.1911, 177.7112, 126.3307]) Cost: 287.936096
Epoch    5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost: 91.371063
Epoch    6/20 hypothesis: tensor([148.1035, 178.0143, 175.3980, 191.0042, 135.7812]) Cost: 29.758249
Epoch    7/20 hypothesis: tensor([150.1744, 180.5042, 177.8509, 193.6753, 137.6805]) Cost: 10.445267
Epoch    8/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost: 4.391237
Epoch    9/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost: 2.493121
Epoch   10/20 hypo

## 3) nn.Module로 구현하는 선형 회귀

- model = nn.Linear(input_dim, output_dim)
- cost = F.mse_loss(prediction, y_train)

### (1) 단순 선형 회귀 구현하기

In [48]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

In [51]:
# 모델 선언 및 초기화. 단순 선형 회귀이므로 input_dim=1, output_dim=1
model = nn.Linear(1,1)

# model에는 가중치 W와 편향 b가 저장되어 있음. model.parameters()로 확인 가능
print(list(model.parameters())) # W = 0.5153, b = -0.4414

[Parameter containing:
tensor([[0.8742]], requires_grad=True), Parameter containing:
tensor([0.3112], requires_grad=True)]


In [52]:
# optimizer 설정, 경사 하강법 이용
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 학습
nb_epochs = 1000
for epoch in range(nb_epochs+1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산, 평균 제곱 오차 함수 이용
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 개선
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward() # backward 연산
    # W와 b를 업데이트
    optimizer.step()

    if epoch % 100 == 0:
    # 100번마다 로그 출력
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

Epoch    0/1000 Cost: 4.610449
Epoch  100/1000 Cost: 0.040473
Epoch  200/1000 Cost: 0.025010
Epoch  300/1000 Cost: 0.015454
Epoch  400/1000 Cost: 0.009550
Epoch  500/1000 Cost: 0.005901
Epoch  600/1000 Cost: 0.003647
Epoch  700/1000 Cost: 0.002253
Epoch  800/1000 Cost: 0.001392
Epoch  900/1000 Cost: 0.000860
Epoch 1000/1000 Cost: 0.000532


In [21]:
# predict 해보기
new_var =  torch.FloatTensor([[4.0]])
pred_y = model(new_var) # forward 연산

print("X = 4일 때의 예측값 :", pred_y.item())

X = 4일 때의 예측값 : 7.891669750213623


In [22]:
# 학습 후 W와 b의 값 -> 학습하며 최적화 된 상태
print(list(model.parameters()))

[Parameter containing:
tensor([[1.9373]], requires_grad=True), Parameter containing:
tensor([0.1426], requires_grad=True)]


### (2) 다중 선형 회귀 구현하기

In [28]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [29]:
# 모델 선언 및 초기화. 다중 선형 회귀이므로 input_dim=3, output_dim=1.
model = nn.Linear(3,1)

print(list(model.parameters())) # W = [0.2975, -0.2548, -0.1119], b = 0.2710

[Parameter containing:
tensor([[ 0.2975, -0.2548, -0.1119]], requires_grad=True), Parameter containing:
tensor([0.2710], requires_grad=True)]


In [30]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 2000
for epoch in range(nb_epochs+1):

    # H(x) 계산
    prediction = model(x_train) # model(x_train)은 model.forward(x_train)와 동일함.

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 개선
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward()
    # W와 b를 업데이트
    optimizer.step()

    if epoch % 100 == 0:
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

Epoch    0/2000 Cost: 31667.597656
Epoch  100/2000 Cost: 0.225993
Epoch  200/2000 Cost: 0.223911
Epoch  300/2000 Cost: 0.221941
Epoch  400/2000 Cost: 0.220059
Epoch  500/2000 Cost: 0.218271
Epoch  600/2000 Cost: 0.216575
Epoch  700/2000 Cost: 0.214950
Epoch  800/2000 Cost: 0.213413
Epoch  900/2000 Cost: 0.211952
Epoch 1000/2000 Cost: 0.210560
Epoch 1100/2000 Cost: 0.209232
Epoch 1200/2000 Cost: 0.207967
Epoch 1300/2000 Cost: 0.206761
Epoch 1400/2000 Cost: 0.205619
Epoch 1500/2000 Cost: 0.204522
Epoch 1600/2000 Cost: 0.203484
Epoch 1700/2000 Cost: 0.202485
Epoch 1800/2000 Cost: 0.201542
Epoch 1900/2000 Cost: 0.200635
Epoch 2000/2000 Cost: 0.199769


In [33]:
# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]])
pred_y = model(new_var)

print("X = [73, 80, 75]일 때의 예측값 :", pred_y.item())

X = [73, 80, 75]일 때의 예측값 : 151.2305450439453


In [36]:
# 학습 후 W와 b의 값 -> 학습하며 최적화 된 상태
print(list(model.parameters()))

[Parameter containing:
tensor([[0.9778, 0.4539, 0.5768]], requires_grad=True), Parameter containing:
tensor([0.2802], requires_grad=True)]


## 4) 클래스로 파이토치 모델 구현하기

### (1) 단순 선형 회귀 모델

In [37]:
class LinearRegressionModel(nn.Module):
    # __init__()에서 모델의 구조와 동작을 정의하는 생성자 정의
    # 객체가 갖는 속성값 초기화 역할, 객체가 생성될 때 자동으로 호출
    def __init__(self):
        super().__init__() # nn.Module 클래스의 속성들을 가지고 초기화
        self.linear = nn.Linear(1, 1) # 단순 선형 회귀이므로 input_dim=1, output_dim=1

    # forward 연산을 진행시키는 함수
    def forward(self, x):
        return self.linear(x)

In [38]:
model = LinearRegressionModel()

### (2) 다중 선형 회귀 모델

In [39]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [40]:
model = MultivariateLinearRegressionModel()

In [41]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화
model = MultivariateLinearRegressionModel()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs+1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost: 19085.046875
Epoch    1/20 Cost: 5982.825195
Epoch    2/20 Cost: 1875.973633
Epoch    3/20 Cost: 588.693237
Epoch    4/20 Cost: 185.199097
Epoch    5/20 Cost: 58.724773
Epoch    6/20 Cost: 19.081615
Epoch    7/20 Cost: 6.655370
Epoch    8/20 Cost: 2.760206
Epoch    9/20 Cost: 1.539075
Epoch   10/20 Cost: 1.156102
Epoch   11/20 Cost: 1.035855
Epoch   12/20 Cost: 0.997936
Epoch   13/20 Cost: 0.985854
Epoch   14/20 Cost: 0.981845
Epoch   15/20 Cost: 0.980371
Epoch   16/20 Cost: 0.979707
Epoch   17/20 Cost: 0.979280
Epoch   18/20 Cost: 0.978941
Epoch   19/20 Cost: 0.978612
Epoch   20/20 Cost: 0.978302


# Lab 04-02 : Loading Data

- dataset = TensorDataset(x_train, y_train)
- dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
    - shuffle=True : 모델이 데이터셋의 순서에 익숙해지는 것을 방지하기 위해 True로 하는 걸 권장

In [74]:
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader    # 데이터로더
from torch.utils.data import Dataset       # Custom Dataset 생성 시 필요한 모듈

## 1) 미니 배치와 데이터 로드(Mini Batch and Data Load)

In [65]:
# TensorDataset은 기본적으로 텐서를 입력으로 받음
x_train  =  torch.FloatTensor([[73,  80,  75],
                               [93,  88,  93],
                               [89,  91,  90],
                               [96,  98,  100],
                               [73,  66,  70]])
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

# TensorDataset의 입력으로 사용하고 dataset으로 저장
dataset = TensorDataset(x_train, y_train)
list(dataset)

[(tensor([73., 80., 75.]), tensor([152.])),
 (tensor([93., 88., 93.]), tensor([185.])),
 (tensor([89., 91., 90.]), tensor([180.])),
 (tensor([ 96.,  98., 100.]), tensor([196.])),
 (tensor([73., 66., 70.]), tensor([142.]))]

In [66]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [71]:
model = nn.Linear(3,1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader):
    # print(batch_idx)
    # print(samples)
    x_train, y_train = samples
    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if (batch_idx+1) % 3 == 0:
        print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, batch_idx+1, len(dataloader),
            cost.item()
            ))

Epoch    0/20 Batch 3/3 Cost: 3870.432373
Epoch    1/20 Batch 3/3 Cost: 125.206253
Epoch    2/20 Batch 3/3 Cost: 3.261635
Epoch    3/20 Batch 3/3 Cost: 0.322391
Epoch    4/20 Batch 3/3 Cost: 1.444060
Epoch    5/20 Batch 3/3 Cost: 0.173145
Epoch    6/20 Batch 3/3 Cost: 0.167353
Epoch    7/20 Batch 3/3 Cost: 0.112946
Epoch    8/20 Batch 3/3 Cost: 1.490191
Epoch    9/20 Batch 3/3 Cost: 1.107029
Epoch   10/20 Batch 3/3 Cost: 1.189956
Epoch   11/20 Batch 3/3 Cost: 0.141519
Epoch   12/20 Batch 3/3 Cost: 0.010687
Epoch   13/20 Batch 3/3 Cost: 1.093170
Epoch   14/20 Batch 3/3 Cost: 0.346136
Epoch   15/20 Batch 3/3 Cost: 1.239259
Epoch   16/20 Batch 3/3 Cost: 0.059322
Epoch   17/20 Batch 3/3 Cost: 0.011253
Epoch   18/20 Batch 3/3 Cost: 0.175285
Epoch   19/20 Batch 3/3 Cost: 0.039143
Epoch   20/20 Batch 3/3 Cost: 0.256743


In [72]:
# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]])
# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var)
print("X = [73, 80, 75]일 때의 예측값 :", pred_y)

X = [73, 80, 75]일 때의 예측값 : tensor([[152.4513]], grad_fn=<AddmmBackward0>)


## 2) 커스텀 데이터셋(Custom Dataset)

In [75]:
# Dataset 상속
class CustomDataset(Dataset):
  def __init__(self):
    self.x_data = [[73, 80, 75],
                   [93, 88, 93],
                   [89, 91, 90],
                   [96, 98, 100],
                   [73, 66, 70]]
    self.y_data = [[152], [185], [180], [196], [142]]

  # 총 데이터의 개수를 리턴
  def __len__(self):
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = torch.FloatTensor(self.y_data[idx])
    return x, y

In [86]:
print(CustomDataset.__len__(dataset))
print(CustomDataset.__getitem__(dataset, 0))

5
(tensor([73., 80., 75.]), tensor([152.]))


In [76]:
dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [77]:
model = torch.nn.Linear(3,1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader):
    # print(batch_idx)
    # print(samples)
    x_train, y_train = samples
    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if (batch_idx+1) % 3 == 0:
        print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, batch_idx+1, len(dataloader),
            cost.item()
            ))

Epoch    0/20 Batch 3/3 Cost: 1622.372559
Epoch    1/20 Batch 3/3 Cost: 34.578045
Epoch    2/20 Batch 3/3 Cost: 10.230608
Epoch    3/20 Batch 3/3 Cost: 8.707434
Epoch    4/20 Batch 3/3 Cost: 5.132642
Epoch    5/20 Batch 3/3 Cost: 6.347807
Epoch    6/20 Batch 3/3 Cost: 0.933870
Epoch    7/20 Batch 3/3 Cost: 9.607294
Epoch    8/20 Batch 3/3 Cost: 7.874491
Epoch    9/20 Batch 3/3 Cost: 4.996592
Epoch   10/20 Batch 3/3 Cost: 5.658033
Epoch   11/20 Batch 3/3 Cost: 2.089722
Epoch   12/20 Batch 3/3 Cost: 7.039704
Epoch   13/20 Batch 3/3 Cost: 1.480743
Epoch   14/20 Batch 3/3 Cost: 8.314655
Epoch   15/20 Batch 3/3 Cost: 4.362920
Epoch   16/20 Batch 3/3 Cost: 6.259235
Epoch   17/20 Batch 3/3 Cost: 9.011814
Epoch   18/20 Batch 3/3 Cost: 7.735606
Epoch   19/20 Batch 3/3 Cost: 6.857420
Epoch   20/20 Batch 3/3 Cost: 0.865852


In [87]:
# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]])
# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var)
print("X = [73, 80, 75]일 때의 예측값 :", pred_y.item())

X = [73, 80, 75]일 때의 예측값 : 154.364990234375


# Lab 05 : Logistic Regression

## 1) 파이토치로 로지스틱 회귀 구현하기

- torch.sigmoid() = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))

In [91]:
# 데이터 준비
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [92]:
# 가중치와 편향 초기화
W = torch.zeros((2, 1), requires_grad=True) # 크기는 2 x 1
b = torch.zeros(1, requires_grad=True)

In [93]:
# sigmoid 함수 생성
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))

# 예측값인 H(x) 출력
print(hypothesis)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)


In [96]:
# 모듈 이용하기
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

print(hypothesis)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)


In [99]:
# 손실함수 생성
losses = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis))
print("각 값들에 대한 오차 :", losses)

cost = losses.mean()
print("MSE :", cost.item())

각 값들에 대한 오차 : tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward0>)
MSE : 0.6931471824645996


In [100]:
# 모듈 이용하기
cost = F.binary_cross_entropy(hypothesis, y_train)

print("MSE :", cost.item())

MSE : 0.6931471824645996


In [101]:
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

# 학습
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = -(y_train * torch.log(hypothesis) +
             (1 - y_train) * torch.log(1 - hypothesis)).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031673
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


In [112]:
# 훈련된 가중치와 편향으로 값 확인
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

print(hypothesis)

# 0.5를 기준으로 0과 1 구분, 1에 가까우면 True/0에 가까우면 False
prediction = hypothesis >= torch.FloatTensor([0.5])

print((prediction))

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01],
        [9.9969e-01]], grad_fn=<SigmoidBackward0>)
tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]])


In [108]:
# 학습 후의 가중치와 편향값 확인
print("W1 :", W[0].item(), "W2 :", W[1].item())
print("b  :", b.item())

W1 : 3.25299072265625 W2 : 1.5178955793380737
b  : -14.481886863708496


## 2) nn.Module로 로지스틱 회귀 구현하기

- nn.Sequential() : nn.Module 층을 차례로 쌓을 수 있도록 해줌.
    - Wx+b와 같은 수식과 시그모이드 함수 등과 같은 여러 함수들을 연결해주는 역할

In [113]:
# 데이터 준비
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [117]:
# 모델 준비
model = nn.Sequential(
   nn.Linear(2, 1), # input_dim = 2, output_dim = 1
   nn.Sigmoid()     # 시그모이드 함수를 거친 후 출력됨
)

model(x_train)

tensor([[0.6620],
        [0.7082],
        [0.5054],
        [0.6447],
        [0.6107],
        [0.4862]], grad_fn=<SigmoidBackward0>)

In [118]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

# 학습
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])  # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == y_train   # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도 계산
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(        # 각 에포크마다 정확도 출력
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/1000 Cost: 0.778947 Accuracy 33.33%
Epoch  100/1000 Cost: 0.134716 Accuracy 100.00%
Epoch  200/1000 Cost: 0.080641 Accuracy 100.00%
Epoch  300/1000 Cost: 0.057899 Accuracy 100.00%
Epoch  400/1000 Cost: 0.045299 Accuracy 100.00%
Epoch  500/1000 Cost: 0.037261 Accuracy 100.00%
Epoch  600/1000 Cost: 0.031672 Accuracy 100.00%
Epoch  700/1000 Cost: 0.027556 Accuracy 100.00%
Epoch  800/1000 Cost: 0.024394 Accuracy 100.00%
Epoch  900/1000 Cost: 0.021888 Accuracy 100.00%
Epoch 1000/1000 Cost: 0.019852 Accuracy 100.00%


In [119]:
# 학습된 모델로 평가
model(x_train)

tensor([[2.7647e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01],
        [9.9969e-01]], grad_fn=<SigmoidBackward0>)

In [129]:
# 학습된 가중치와 편항값 확인
print(list(model.parameters()))

[Parameter containing:
tensor([[3.2530, 1.5179]], requires_grad=True), Parameter containing:
tensor([-14.4819], requires_grad=True)]


## 3) 클래스로 파이토치 모델 구현하기

In [132]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [133]:
# 데이터 준비
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

# 모델 준비
model = BinaryClassifier()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

# 학습
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])  # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == y_train   # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/1000 Cost: 0.576740 Accuracy 83.33%
Epoch  100/1000 Cost: 0.134897 Accuracy 100.00%
Epoch  200/1000 Cost: 0.080704 Accuracy 100.00%
Epoch  300/1000 Cost: 0.057931 Accuracy 100.00%
Epoch  400/1000 Cost: 0.045319 Accuracy 100.00%
Epoch  500/1000 Cost: 0.037274 Accuracy 100.00%
Epoch  600/1000 Cost: 0.031682 Accuracy 100.00%
Epoch  700/1000 Cost: 0.027563 Accuracy 100.00%
Epoch  800/1000 Cost: 0.024400 Accuracy 100.00%
Epoch  900/1000 Cost: 0.021893 Accuracy 100.00%
Epoch 1000/1000 Cost: 0.019856 Accuracy 100.00%


In [134]:
# 학습된 가중치와 편항값 확인
print(list(model.parameters()))

[Parameter containing:
tensor([[3.2528, 1.5178]], requires_grad=True), Parameter containing:
tensor([-14.4811], requires_grad=True)]


In [141]:
# 임의의 입력 [4, 5]를 선언
new_var =  torch.FloatTensor([[4, 5]])
pred_y = model(new_var)

print("X = [4, 5]일 때의 예측값 :", int(pred_y.item() >= torch.FloatTensor([0.5])))

X = [4, 5]일 때의 예측값 : 1
